In [1]:
import requests
from scrapy.selector import Selector
from tqdm import tqdm
from nested_lookup import nested_lookup
import re
import time
import pandas as pd

In [2]:
url = "https://www.tyres-online.in/bridgestone-turanza-er-300-205-55-r16-91h--7041149"

In [3]:
res = requests.get(url)
res

<Response [200]>

In [4]:
sel = Selector(res)
sel

<Selector xpath=None data='<html class="no-js" lang="en"> <!--<!...'>

In [5]:
title_name = sel.xpath('//div[@class="product-header tablet-up"]//h1[@class="product-title sl-product-details-title"]//text()').extract()
title_name = title_name[0].strip()
title_name

'Bridgestone Turanza ER 300 205/55 R16 91H'

In [6]:
best_price = sel.xpath('//div[@class="product-max-price hidden"]//text()').extract()
best_price

['7500.0']

In [7]:
spacification_key = sel.xpath('//span[@class="product-specifications-label"]//text()')[:10].extract()
spacification_key

['Vehicle type:',
 'Usage:',
 'Manufacturer:',
 'Width:',
 'Height:',
 'Construction:',
 'Diameter:',
 'Load index:',
 'Speed index:',
 'EAN:']

In [8]:
spacification_value = sel.xpath('//div[@class="product-specifications-value"]//span//text()')[:10].extract()
spacification_value

['Car',
 'Summer',
 'Bridgestone',
 '205',
 '55',
 'R',
 '16',
 '91',
 'H',
 '3286340552912']

In [9]:
spacification = {}
for spacification_ke,spacification_valu in zip(spacification_key,spacification_value):
    spacification[spacification_ke] ="".join(spacification_valu)
    
print(spacification)    

{'Vehicle type:': 'Car', 'Usage:': 'Summer', 'Manufacturer:': 'Bridgestone', 'Width:': '205', 'Height:': '55', 'Construction:': 'R', 'Diameter:': '16', 'Load index:': '91', 'Speed index:': 'H', 'EAN:': '3286340552912'}


In [10]:
img_link = sel.xpath('//img[@class="product__picture product-image-thumbnail"]/@srcset').extract()
img_link

['https://static.toroleo.de/s/content-synced/p30/59/83059_1530308300930_thumb.jpg 30w, https://static.toroleo.de/s/content-synced/p30/59/83059_1530308300930_small.jpg 96w, https://static.toroleo.de/s/content-synced/p30/59/83059_1530308300930_medium.jpg 135w, https://static.toroleo.de/s/content-synced/p30/59/83059_1530308300930_big.jpg 200w',
 'https://static.toroleo.de/s/content-synced/pp0/21/21_bridgestone_turanza_er_300_thumb.jpg 30w, https://static.toroleo.de/s/content-synced/pp0/21/21_bridgestone_turanza_er_300_small.jpg 96w, https://static.toroleo.de/s/content-synced/pp0/21/21_bridgestone_turanza_er_300_medium.jpg 135w, https://static.toroleo.de/s/content-synced/pp0/21/21_bridgestone_turanza_er_300_big.jpg 200w']

In [11]:
imgs_url = {}
img_link = sel.xpath('//img[@class="product__picture product-image-thumbnail"]/@srcset').extract()
for idx,im_url in enumerate(img_link):
    i_url ='img_url{}'
    nimg_url = [i_url.format(idx)]
    img_url = [im_url]
#     print(nimg_url)
#     print(img_url)
    for im_url,img_url in zip(nimg_url,img_url):
        imgs_url[im_url] = "".join(img_url)
        
print(imgs_url)        

{'img_url0': 'https://static.toroleo.de/s/content-synced/p30/59/83059_1530308300930_thumb.jpg 30w, https://static.toroleo.de/s/content-synced/p30/59/83059_1530308300930_small.jpg 96w, https://static.toroleo.de/s/content-synced/p30/59/83059_1530308300930_medium.jpg 135w, https://static.toroleo.de/s/content-synced/p30/59/83059_1530308300930_big.jpg 200w', 'img_url1': 'https://static.toroleo.de/s/content-synced/pp0/21/21_bridgestone_turanza_er_300_thumb.jpg 30w, https://static.toroleo.de/s/content-synced/pp0/21/21_bridgestone_turanza_er_300_small.jpg 96w, https://static.toroleo.de/s/content-synced/pp0/21/21_bridgestone_turanza_er_300_medium.jpg 135w, https://static.toroleo.de/s/content-synced/pp0/21/21_bridgestone_turanza_er_300_big.jpg 200w'}


In [12]:
urls ='''https://www.tyres-online.in/bridgestone-turanza-er-300-205-55-r16-91h--7041149
https://www.tyres-online.in/bridgestone-turanza-t001-215-55-r17-94v--10649755
https://www.tyres-online.in/continental-contisportcontact-5-255-35-r18-94y--11825320
https://www.tyres-online.in/pirelli-p-zero-runflat-245-50-r18-100y--15233227
https://www.tyres-online.in/pirelli-cinturato-p7-225-45-r17-91v--8224867
https://www.tyres-online.in/michelin-primacy-3-215-50-r17-95w--6254768
https://www.tyres-online.in/continental-contisportcontact-5-ssr-225-50-r18-95w--6058044
https://www.tyres-online.in/michelin-primacy-hp-225-55-r16-95w--13201547
https://www.tyres-online.in/continental-contipremiumcontact-2-225-55-r16-95y--11497679
https://www.tyres-online.in/bridgestone-turanza-er-300-2-rft-195-55-r16-87v--7893158
https://www.tyres-online.in/continental-contisportcontact-3-e-ssr-275-40-r18-99y--5341368
https://www.tyres-online.in/bridgestone-turanza-er-300-ecopia-225-50-r16-92v--5734553
https://www.tyres-online.in/pirelli-p-zero-275-45-zr18-103y--1077499
https://www.tyres-online.in/michelin-pilot-sport-ps2-295-30-zr19-100y--15163553
https://www.tyres-online.in/bridgestone-potenza-re-050-a-225-45-r17-91w--8220879
https://www.tyres-online.in/michelin-pilot-sport-ps2-255-40-r19-100y--9396351
https://www.tyres-online.in/bridgestone-turanza-er-300-rft-205-55-r16-91v--291070
https://www.tyres-online.in/pirelli-p-zero-275-40-zr19-105y--13594840
https://www.tyres-online.in/pirelli-p-zero-245-45-zr19-102y--4354073
https://www.tyres-online.in/pirelli-p-zero-235-40-zr18-95y--2584774
https://www.tyres-online.in/continental-contisportcontact-5-225-50-r17-94y--9470072
https://www.tyres-online.in/pirelli-cinturato-p7-runflat-225-55-r17-97y--8745176
https://www.tyres-online.in/pirelli-cinturato-p7-runflat-225-50-r17-94w--3694757
https://www.tyres-online.in/bridgestone-turanza-t001-ext-225-50-r17-94w--5271742
https://www.tyres-online.in/dunlop-sp-sport-01-225-50-r17-94y--11821280
https://www.tyres-online.in/bridgestone-potenza-re-050-a-245-45-r17-99y--11956236
https://www.tyres-online.in/bridgestone-potenza-re-050-a-225-50-r17-94y--11759814
https://www.tyres-online.in/bridgestone-potenza-s001-245-45-r19-98y--11169982
https://www.tyres-online.in/michelin-primacy-3-zp-225-50-r17-94w--1933384
https://www.tyres-online.in/pirelli-cinturato-p7-runflat-245-50-r18-100w--4026555
https://www.tyres-online.in/pirelli-cinturato-p7-225-45-r17-91v--14315749
https://www.tyres-online.in/pirelli-cinturato-p7-225-45-r17-91y--8351928
https://www.tyres-online.in/continental-contisportcontact-3-265-35-r18-97y--5730497
https://www.tyres-online.in/michelin-primacy-hp-235-55-r17-99w--10186948
https://www.tyres-online.in/dunlop-sport-maxx-rt-225-45-r17-91y--13926470
https://www.tyres-online.in/continental-contisportcontact-5-245-40-r17-91w--6447264
https://www.tyres-online.in/continental-contisportcontact-5-225-45-r17-91w--14835872
https://www.tyres-online.in/continental-contisportcontact-5-225-45-r17-91v--14118945
https://www.tyres-online.in/continental-contisportcontact-3-245-40-r18-97y--3833912
https://www.tyres-online.in/bridgestone-turanza-er-300-245-45-r18-100y--9535574
https://www.tyres-online.in/continental-contisportcontact-5-245-40-r18-97y--13598861
https://www.tyres-online.in/continental-contisportcontact-5-245-40-r18-97y--12087520
https://www.tyres-online.in/continental-contisportcontact-5-225-45-r17-91y--14119080
https://www.tyres-online.in/continental-contiecocontact-5-205-55-r16-91v--6647841
https://www.tyres-online.in/pirelli-p-zero-305-30-zr19-102y--225292
https://www.tyres-online.in/continental-contiecocontact-5-ssr-225-55-r16-95v--14774527
https://www.tyres-online.in/continental-contipremiumcontact-5-205-55-r16-91w--12939502
https://www.tyres-online.in/bridgestone-potenza-s001-ext-245-45-r19-102y--5533756
https://www.tyres-online.in/michelin-pilot-sport-3-235-40-zr18-95y--13066401
https://www.tyres-online.in/bridgestone-potenza-re-050-a-rft-255-40-r17-94v--9404621
https://www.tyres-online.in/michelin-primacy-3-zp-245-50-r18-100w--884951
https://www.tyres-online.in/continental-contisportcontact-3-ssr-245-40-r18-93y--9142357
https://www.tyres-online.in/bridgestone-potenza-re-050-rft-225-50-r17-94w--5070890
https://www.tyres-online.in/dunlop-sp-sport-maxx-gt-245-45-zr19-102y--8744986
https://www.tyres-online.in/continental-contisportcontact-3-235-40-zr18-95y--16277694
https://www.tyres-online.in/pirelli-p-zero-305-30-zr20-103y--618543
https://www.tyres-online.in/pirelli-p-zero-255-40-r19-100y--1011934
https://www.tyres-online.in/pirelli-p-zero-235-35-zr19-91y--1536204
https://www.tyres-online.in/pirelli-p-zero-235-55-r18-104y--3567832
https://www.tyres-online.in/bridgestone-potenza-re-050-a-ecopia-245-40-r18-97y--4227086
https://www.tyres-online.in/bridgestone-turanza-t001-225-45-r17-91v--16154829
https://www.tyres-online.in/dunlop-sp-sport-maxx-gt-255-35-r18-94y--946188
https://www.tyres-online.in/michelin-pilot-sport-3-255-35-zr18-94y--8413345
https://www.tyres-online.in/michelin-primacy-hp-245-45-r17-95y--1011724
https://www.tyres-online.in/bridgestone-potenza-s001-245-45-r17-95w--4685894
https://www.tyres-online.in/dunlop-sport-maxx-rt-245-35-zr19-93y--5079128
https://www.tyres-online.in/dunlop-sp-sport-maxx-gt-235-40-zr18-95y--7176277
https://www.tyres-online.in/continental-contisportcontact-5p-235-45-zr19-99y--7569612
https://www.tyres-online.in/michelin-pilot-sport-3-245-45-zr17-99y--6971489
https://www.tyres-online.in/michelin-pilot-sport-3-235-45-zr19-99w--16675067
https://www.tyres-online.in/michelin-primacy-3-225-55-r17-97y--16609464
https://www.tyres-online.in/pirelli-p-zero-runflat-275-40-r19-101y--6189182
https://www.tyres-online.in/pirelli-p-zero-245-50-zr18-100y--8483003
https://www.tyres-online.in/pirelli-p-zero-295-30-zr19-100y--13791244
https://www.tyres-online.in/pirelli-p-zero-265-35-zr20-95y--12152879
https://www.tyres-online.in/pirelli-p-zero-265-40-r20-104y--15626462
https://www.tyres-online.in/pirelli-p-zero-255-45-zr19-100y--10453130
https://www.tyres-online.in/pirelli-p-zero-265-35-r18-97y--9335000
https://www.tyres-online.in/pirelli-p-zero-245-40-r18-97y--9789474
https://www.tyres-online.in/pirelli-p-zero-275-35-zr20-102y--4681926
https://www.tyres-online.in/pirelli-p-zero-245-50-zr18-100y--8483003
https://www.tyres-online.in/pirelli-p-zero-265-40-r20-104y--15626462
https://www.tyres-online.in/pirelli-p-zero-295-30-zr19-100y--13791244
https://www.tyres-online.in/pirelli-p-zero-265-35-zr20-95y--12152879
https://www.tyres-online.in/pirelli-p-zero-pz4-sc-245-40-zr19-98y--15958157
https://www.tyres-online.in/pirelli-p-zero-pz4-sc-235-35-zr20-88y--2195501
https://www.tyres-online.in/michelin-primacy-3-zp-245-50-r18-100w--884951
https://www.tyres-online.in/pirelli-p-zero-255-45-r19-100y--15233199
https://www.tyres-online.in/continental-contisportcontact-5p-235-40-zr18-95y--15818912
https://www.tyres-online.in/yokohama-c.-drive-2-ac02a-205-55-r16-91h--15954097
https://www.tyres-online.in/yokohama-c.-drive-2-ac02a-235-50-r18-97v--16285914
https://www.tyres-online.in/yokohama-bluearth-ae01--205-60-r15-91h--15958162
https://www.tyres-online.in/yokohama-advan-fleva-v701-255-40-r17-94w--2592993
https://www.tyres-online.in/yokohama-c.-drive-2-ac02a-zps-235-50-r18-97v--3768538
https://www.tyres-online.in/hankook-ventus-s1-evo-2-k117-225-50-zr17-94w--4423704
https://www.tyres-online.in/goodyear-efficientgrip-performance-rof-225-50-r17-94w--5144770
https://www.tyres-online.in/goodyear-eagle-f1-asymmetric-2-rof-275-35-r20-102y--225340
https://www.tyres-online.in/yokohama-advan-sport-v105--225-45-r17-91w--9793713
https://www.tyres-online.in/hankook-ventus-s1-evo-2-k117-225-50-r17-94y--4554985
https://www.tyres-online.in/yokohama-advan-a052-245-40-r18-97y--14323726
https://www.tyres-online.in/goodyear-efficientgrip-performance-rof-225-50-r17-94w--5144770
https://www.tyres-online.in/goodyear-excellence-rof-225-55-r17-97y--4223024
https://www.tyres-online.in/yokohama-advan-a052-245-40-r18-97y--14323726'''.split('\n')
urls = set(urls)
len(urls)

96

In [13]:
items = []
for url in tqdm(urls):
    res = requests.get(url)
    sel = Selector(res)
    
    item ={}
    item['Prduct_url'] = url
    item['title_name'] = sel.xpath('//div[@class="product-header tablet-up"]//h1[@class="product-title sl-product-details-title"]//text()').extract()[0].strip()
    item['best_price'] = sel.xpath('//div[@class="product-max-price hidden"]//text()').extract()
    
    spacification_key = sel.xpath('//span[@class="product-specifications-label"]//text()').extract()
    spacification_value = sel.xpath('//div[@class="product-specifications-value"]//span//text()').extract()
    spacification = {}
    for spacification_ke,spacification_valu in zip(spacification_key,spacification_value):
        spacification[spacification_ke] ="".join(spacification_valu)
#     print(spacification)    
    
    imgs_url = {}
    img_link = sel.xpath('//img[@class="product__picture product-image-thumbnail"]/@srcset').extract()
    for idx,im_url in enumerate(img_link):
        i_url ='img_url{}'
        nimg_url = [i_url.format(idx)]
        img_url = [im_url]
    
        for im_url,img_url in zip(nimg_url,img_url):
            imgs_url[im_url] = "".join(img_url)

#     print(imgs_url)        
    
    
    
    items.append(item)
    item.update(spacification)
    item.update(item)
    
items    
    
   

100%|██████████████████████████████████████████████████████████████████████████████████| 96/96 [02:04<00:00,  1.30s/it]


[{'Prduct_url': 'https://www.tyres-online.in/hankook-ventus-s1-evo-2-k117-225-50-zr17-94w--4423704',
  'title_name': 'Hankook Ventus S1 Evo 2 K117 225/50 ZR17 94W',
  'best_price': ['11500.0'],
  'Vehicle type:': 'Car',
  'Usage:': 'Summer',
  'Manufacturer:': 'Hankook',
  'Width:': '225',
  'Height:': '50',
  'Construction:': 'ZR',
  'Diameter:': '17',
  'Load index:': '94',
  'Speed index:': 'W',
  'EAN:': '8808563330365',
  'EU tyre label:': 'C'},
 {'Prduct_url': 'https://www.tyres-online.in/continental-contiecocontact-5-205-55-r16-91v--6647841',
  'title_name': 'Continental CONTIECOCONTACT 5 205/55 R16 91V',
  'best_price': ['11000.0'],
  'Vehicle type:': 'Car',
  'Usage:': 'Summer',
  'Manufacturer:': 'Continental',
  'Width:': '205',
  'Height:': '55',
  'Construction:': 'R',
  'Diameter:': '16',
  'Load index:': '91',
  'Speed index:': 'V',
  'EAN:': '4019238525984',
  'Customer reviews:': 'B',
  'EU tyre label:': 'B'},
 {'Prduct_url': 'https://www.tyres-online.in/pirelli-p-zero

In [14]:
df = pd.DataFrame(items)
df

,Prduct_url,title_name,best_price,Vehicle type:,Usage:,Manufacturer:,Width:,Height:,Construction:,Diameter:,Load index:,Speed index:,EAN:,EU tyre label:,Customer reviews:,Runflat:
0,https://www.tyres-online.in/hankook-ventus-s1-...,Hankook Ventus S1 Evo 2 K117 225/50 ZR17 94W,[11500.0],Car,Summer,Hankook,225,50,ZR,17,94,W,8808563330365,C,NaN,NaN
1,https://www.tyres-online.in/continental-contie...,Continental CONTIECOCONTACT 5 205/55 R16 91V,[11000.0],Car,Summer,Continental,205,55,R,16,91,V,4019238525984,B,B,NaN
2,https://www.tyres-online.in/pirelli-p-zero-245...,Pirelli P ZERO 245/50 ZR18 100Y,[19000.0],Car,Summer,Pirelli,245,50,ZR,18,100,Y,8019227231571,A,D,NaN
3,https://www.tyres-online.in/goodyear-excellenc...,Goodyear Excellence ROF 225/55 R17 97Y,[14500.0],Car,Summer,Goodyear,225,55,R,17,97,Y,5452000651723,B,D,Runflat
4,https://www.tyres-online.in/continental-contie...,Continental ContiEcoContact 5 SSR 225/55 R16 95V,[15500.0],Car,Summer,Continental,225,55,R,16,95,V,4019238588613,C,NaN,Runflat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,https://www.tyres-online.in/bridgestone-potenz...,Bridgestone Potenza RE 050 A 225/45 R17 91W,[13000.0],Car,Summer,Bridgestone,225,45,R,17,91,W,3286340295819,B,F,NaN
92,https://www.tyres-online.in/continental-contis...,Continental ContiSportContact 5 245/40 R17 91W,[21500.0],Car,Summer,Continental,245,40,R,17,91,W,4019238456233,B,C,NaN
93,https://www.tyres-online.in/yokohama-advan-fle...,Yokohama Advan Fleva V701 255/40 R17 94W,[759.0],Car,Summer,Yokohama,255,40,R,17,94,W,4968814900106,A,C,NaN
94,https://www.tyres-online.in/pirelli-p-zero-265...,Pirelli P ZERO 265/35 R18 97Y,[27500.0],Car,Summer,Pirelli,265,35,R,18,97,Y,8019227194951,B,C,NaN


In [15]:
df.to_csv('CarSummerTyres.csv')